In [37]:
import pandas as pd
import json
import ast
import pyarrow.parquet as pq

In [38]:
#Lista para almacenar los diccionarios JSON de cada línea
review = []

#Ruta del archivo JSON
file_path = 'Datasets\\JSON\\australian_user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            review.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
data_user_reviews = pd.DataFrame(review)

In [39]:
#Seguimos con la desanidacion de la estructura del Dataframe
data_user_review1 = data_user_reviews.explode(['reviews'])
data_user_review2 = data_user_review1['reviews'].apply(pd.Series)
data_user_review3 = pd.concat([data_user_review1, data_user_review2], axis=1)
data_user_reviews = data_user_review3

In [40]:
#Realizamos la eliminacion de las columnas que no necesitamos
df_clean_reviews = data_user_reviews.drop('funny', axis=1)
df_clean_reviews = df_clean_reviews.drop('last_edited', axis=1)
df_clean_reviews = df_clean_reviews.drop('user_url', axis=1)
df_clean_reviews = df_clean_reviews.drop('reviews', axis=1)
df_clean_reviews = df_clean_reviews.drop('helpful', axis=1)
df_clean_reviews = df_clean_reviews.drop(df_clean_reviews.columns[-1], axis=1)
data_user_reviews = df_clean_reviews

#Eliminamos los valores "Nulos" de las columnas que nos interesan
df_clean_reviews = data_user_reviews.dropna(subset=['posted'])
df_clean_reviews = df_clean_reviews.dropna(subset=['recommend'])
df_clean_reviews = df_clean_reviews.dropna(subset=['review'])
df_clean_reviews = df_clean_reviews.dropna(subset=['item_id'])
data_user_reviews = df_clean_reviews

In [41]:
import nltk
# descarga del diccionario vader_lexicon 
nltk.download('vader_lexicon')
# importar el analizador de sentimientos vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [42]:
#Inicializa el analizador de sentimiento de NLTK
sia = SentimentIntensityAnalyzer()

#Definir una funcion para categorizar el sentimiento
def categorize_sentiment(score):
    if score < -0.05:
        return 0 #Malo
    elif score > 0.05:
        return 2 #Positivo
    else:
        return 1 #Neutral

#Convertir la columna 'review' en string
data_user_reviews['review'] = data_user_reviews['review'].astype(str) 
#Aplica el analisis de sentimientos a las reseñas y categoriza el resultado
data_user_reviews['review'] = data_user_reviews['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
data_user_reviews['review'] = data_user_reviews['review'].apply(categorize_sentiment)

In [43]:
#Extraer el año de la columna 'posted' y almacenarlo en una nueva columna 'year_posted'
data_user_reviews['year_posted'] = data_user_reviews['posted'].str.extract(r'(\d{4})')
#Eliminar la columna 'posted' del DataFrame data_user_review
data_user_reviews.drop('posted' , axis = 1, inplace = True) 

In [44]:
#Imputamos valores para la columna year_posted, pues son 22% de los datos en total.
#Imputamos los valores nulos de la columna 'year_posted' con el valor 2013 que es el valor de la mediana
data_user_reviews['year_posted'].fillna(2013, inplace=True)

In [45]:
#Eliminamos valores duplicados
data_user_reviews = data_user_reviews.drop_duplicates()

In [46]:
#Convertimos el tipo de variable de 'year_posted' a string
data_user_reviews['year_posted'] = data_user_reviews['year_posted'].astype(str)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1776\2423273455.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_user_reviews['year_posted'] = data_user_reviews['year_posted'].astype(str)


In [48]:
data_user_reviews.to_parquet('DatasetsLimpios\\user_reviews.parquet')